In [ ]:
import numpy as np
from PIL import Image
import json
import os

# Fonction pour générer une fractale de Julia (exemple simplifié)
def generate_julia(c_real, c_imag, max_iterations, threshold, width, height):
    # Convertir les paramètres en types Python natifs si nécessaire
    c = complex(c_real, c_imag)
    
    # Créer une image vide
    image = np.zeros((height, width), dtype=np.uint8)

    # Définir la région du plan complexe à visualiser
    x_min, x_max = -2.0, 2.0
    y_min, y_max = -2.0, 2.0

    for y in range(height):
        for x in range(width):
            # Mapper les coordonnées de pixel au plan complexe
            zx = x_min + (x / width) * (x_max - x_min)
            zy = y_min + (y / height) * (y_max - y_min)
            z = complex(zx, zy)
            
            iteration = 0
            while abs(z) < threshold and iteration < max_iterations:
                z = z*z + c # Équation de Julia
                iteration += 1
            
            # Normaliser l'itération pour la couleur (exemple simple)
            color = int(255 * (iteration / max_iterations))
            image[y, x] = color
            
    return image

# Paramètres de génération de la base de données
NUM_SAMPLES = 10000
IMAGE_SIZE = (128, 128) # Largeur, Hauteur
OUTPUT_DIR = "fractal_dataset"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

metadata = []

for i in range(NUM_SAMPLES):
    # Générer des paramètres aléatoires pour Julia
    c_real = np.random.uniform(-1.5, 1.5)
    c_imag = np.random.uniform(-1.5, 1.5)
    max_iter = np.random.randint(50, 500)
    threshold = np.random.uniform(2.0, 10.0) # On peut aussi varier le seuil
    
    # Vous pouvez ajouter d'autres paramètres ici et les passer à la fonction de génération
    # p_real = np.random.uniform(1.0, 3.0) 
    # p_imag = np.random.uniform(-1.0, 1.0)

    # Générer l'image
    fractal_image_array = generate_julia(c_real, c_imag, max_iter, threshold, IMAGE_SIZE[0], IMAGE_SIZE[1])
    
    # Sauvegarder l'image
    image_filename = f"fractal_{i:05d}.png"
    image_path = os.path.join(OUTPUT_DIR, image_filename)
    Image.fromarray(fractal_image_array).save(image_path)
    
    # Enregistrer les métadonnées (paramètres)
    metadata.append({
        "id": i,
        "image_path": image_path,
        "params": {
            "c_real": c_real,
            "c_imag": c_imag,
            "max_iterations": max_iter,
            "threshold": threshold,
            # "p_real": p_real,
            # "p_imag": p_imag
        }
    })

# Sauvegarder toutes les métadonnées dans un fichier JSON
with open(os.path.join(OUTPUT_DIR, "metadata.json"), "w") as f:
    json.dump(metadata, f, indent=4)

print(f"Génération de {NUM_SAMPLES} fractales terminée dans {OUTPUT_DIR}/")

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Conv2DTranspose, LeakyReLU, BatchNormalization, Activation

def build_fractal_generator(input_params_dim, image_width, image_height, channels=1):
    input_layer = Input(shape=(input_params_dim,))

    # Transformation des paramètres en une représentation spatiale initiale
    x = Dense(4 * 4 * 256)(input_layer) # 4x4 image, 256 filtres
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((4, 4, 256))(x)

    # Déconvolution pour augmenter la résolution
    # 4x4x256 -> 8x8x128
    x = Conv2DTranspose(128, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 8x8x128 -> 16x16x64
    x = Conv2DTranspose(64, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 16x16x64 -> 32x32x32
    x = Conv2DTranspose(32, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    # Si IMAGE_SIZE est 128x128
    # 32x32x32 -> 64x64x16
    x = Conv2DTranspose(16, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 64x64x16 -> 128x128xchannels
    x = Conv2DTranspose(channels, kernel_size=5, strides=2, padding='same')(x)
    output_layer = Activation('sigmoid')(x) # Pour des pixels entre 0 et 1

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Exemple d'utilisation
# input_dim = len(vos_parametres) # 5 à 10
# generator = build_fractal_generator(input_dim, IMAGE_SIZE[0], IMAGE_SIZE[1])
# generator.summary()